In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("I5S_labeled.csv")

In [3]:
df.head()

,Unnamed: 0,Date,Time,Station,TotalFlow,AvgOccupancy,AvgSpeed,Order,Class
0,0,01/01/2015,00:00:00,1108417,76.0,0.0125,69.6,73,0.0
1,1,01/01/2015,00:05:00,1108417,92.0,0.0150,69.2,73,0.0
2,2,01/01/2015,00:10:00,1108417,100.0,0.0162,69.8,73,0.0
3,3,01/01/2015,00:15:00,1108417,91.0,0.0143,69.9,73,0.0
4,4,01/01/2015,00:20:00,1108417,176.0,0.0291,67.6,73,0.0


In [4]:
df.groupby('Class').count()

,Unnamed: 0,Date,Time,Station,TotalFlow,AvgOccupancy,AvgSpeed,Order
Class,,,,,,,,
0.0,9998289,9998289,9998289,9998289,9998289,9998289,9998289,9998289
0.5,585880,585880,585880,585880,585880,585880,585880,585880
1.0,189009,189009,189009,189009,189009,189009,189009,189009


## Pivot Class to create Naive Classifier

In [7]:
df2 = pd.pivot_table(df,index='Station', columns='Time', values='Class')

In [15]:
df['conc'] = df['Station'].astype(str)+'__'+df['Date'].astype(str)

In [16]:
df2 = df.pivot(index='conc', columns='Time', values='Class')

In [17]:
df2.shape

(37526, 288)

In [20]:
df2 = df2.reset_index()

In [28]:
df2['Station'], df2['Date'] = df2['conc'].str.split('__', 1).str

In [35]:
avgs = df2.groupby('Station').mean().reset_index()

In [41]:
result = pd.melt(avgs, id_vars=['Station'], value_vars=list(avgs.columns[1:]), var_name='Time', value_name='Percent')

In [44]:
result['Station'] = result['Station'].astype(int)

In [50]:
sum(result['Percent']>.1) / float(result.shape[0]) # percent that are above threshold

0.14370954692556634

In [51]:
def regular_irregular(row, threshold = .1):
    if row['Class'] == 0:
        return 'No Jam'
    elif row['Percent'] < threshold:
        return 'Irregular'
    else:
        return 'Regular'

In [52]:
df['Regular'] = df.merge(result).apply(lambda row: regular_irregular(row), axis=1)

In [56]:
df.merge(result).to_csv('I5S_labeled_w_regular.csv')